In [ ]:
%pip install redis

In [ ]:
import redis
import numpy as np
import torch
from urllib.parse import urlparse


In [ ]:
#https://redis.io/docs/latest/develop/get-started/

dtype_np = np.float32
dtype_pt = torch.float32
device_embeddings = 'cpu'


In [ ]:
# url='redis://localhost:6379?db=1'
# redis_client = redis.from_url(url) # redis.Redis(host=host, port=port, db=dbid)
redis_client = redis.Redis(host="localhost", port=6379, db=1, decode_responses=False)

In [ ]:
try:
    res = redis_client.ping()
    print(res)
except redis.exceptions.ConnectionError as err:
    print(err)


In [ ]:
def __bytes_to_tensor__(tensor_as_byte):
    if tensor_as_byte is None:
        return None
    tensor_np = np.frombuffer(tensor_as_byte, dtype=dtype_np)
    tensor_pt = torch.as_tensor(tensor_np, dtype=dtype_pt, device=device_embeddings)
    return tensor_pt

def __tensor_to_bytes__(tensor):
    if tensor is None:
        return None
    tensor_np = tensor.cpu().numpy().astype(dtype_np)
    tensor_bytes = tensor_np.tobytes()
    return tensor_bytes

In [ ]:
a = torch.rand(int(1e4),5)
print(a.size())

In [ ]:
# add data
item_dict = {key: __tensor_to_bytes__(a[i]) for i, key in enumerate(range(a.shape[0]))}
res = redis_client.mset(item_dict)
print(res)

In [ ]:
# retrieve
keys = [987340132, 1,2,3,4,42, 314123412]
values = redis_client.mget(keys)
tensors = [ __bytes_to_tensor__(value) for value in values if value is not None ]
tensors = torch.stack(tensors)
print(tensors.size())


In [ ]:
# contains
keys = [12,34,4,54,2398641923]
contains = map(redis_client.__contains__, keys)
print(list(contains))

In [ ]:
# delete all keys from current DB
redis_client.flushdb()

In [ ]:
# delete all keys from all DBs
redis_client.flushall()